In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 61.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=68c41ee48e50d893e4a1c18b167b8258d82facd5eb3d22fdc0d60a6537ba746b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
movies = files.upload()

Saving movies.csv to movies.csv


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
.appName("trial")  \
.config('spark.ui.port','4050') \
.getOrCreate()

In [4]:
spark

In [5]:
dataframe = spark.read.csv('movies.csv')
dataframe.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [6]:
dataframe = spark.read.format("csv").option("header","false").load("movies.csv")
dataframe.show()

+--------------------+--------------------+----+
|                 _c0|                 _c1| _c2|
+--------------------+--------------------+----+
|1::Toy Story (199...|                null|null|
|2::Jumanji (1995)...|                null|null|
|3::Grumpier Old M...|                null|null|
|4::Waiting to Exh...|                null|null|
|5::Father of the ...|                null|null|
|6::Heat (1995)::A...|                null|null|
|7::Sabrina (1995)...|                null|null|
|8::Tom and Huck (...|                null|null|
|9::Sudden Death (...|                null|null|
|10::GoldenEye (19...|                null|null|
|11::American Pres...| The (1995)::Come...|null|
|12::Dracula: Dead...|                null|null|
|13::Balto (1995):...|                null|null|
|14::Nixon (1995):...|                null|null|
|15::Cutthroat Isl...|                null|null|
|16::Casino (1995)...|                null|null|
|17::Sense and Sen...|                null|null|
|18::Four Rooms (1..

In [7]:
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [10]:
schema = StructType([
    StructField("Id",IntegerType(),True), \
    StructField("movie_name",StringType(),True), \
    StructField("movies genre",StringType(),True),
                     ])

In [11]:
df=spark.read.csv("movies.csv",header = True , sep="::",schema=schema)
df.show(truncate = False)

+---+-------------------------------------+------------------------------+
|Id |movie_name                           |movies genre                  |
+---+-------------------------------------+------------------------------+
|2  |Jumanji (1995)                       |Adventure|Children's|Fantasy,,|
|3  |Grumpier Old Men (1995)              |Comedy|Romance,,              |
|4  |Waiting to Exhale (1995)             |Comedy|Drama,,                |
|5  |Father of the Bride Part II (1995)   |Comedy,,                      |
|6  |Heat (1995)                          |Action|Crime|Thriller,,       |
|7  |Sabrina (1995)                       |Comedy|Romance,,              |
|8  |Tom and Huck (1995)                  |Adventure|Children's,,        |
|9  |Sudden Death (1995)                  |Action,,                      |
|10 |GoldenEye (1995)                     |Action|Adventure|Thriller,,   |
|11 |American President, The (1995)       |Comedy|Drama|Romance,         |
|12 |Dracula: Dead and Lo

In [16]:

df = combined.select('movie_name')
df2 = df.withcolumn('name'.split(col('movie_name'),' ').getItem(0)).show()


NameError: ignored

In [ ]:
df1 = spark.read.format('csv').option("header","true").option("multiline","true").load("movies.csv")
df1.show()